# Testing Chatbot

**Importing the Libraries Needed**

In [28]:
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain.chains import RetrievalQA
from collections import Counter
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
from langchain_openai import ChatOpenAI
from textblob import TextBlob
from datetime import datetime, timedelta
from secret_key import openai_key
import re
import faiss
from tqdm import tqdm
import torch
from langchain.schema import Document
import os
import pandas as pd

**Setting up the LLM Used**

In [3]:
# API Key for OpenAI
os.environ['OPENAI_API_KEY'] = openai_key

# Set up the OpenAI chat model
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.4)
# llm = ChatOpenAI(temperature=0.3, model="gpt-4o", max_tokens=1000)

C:\Users\colaz\AppData\Local\Temp\ipykernel_9092\2534856997.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.4)


In [3]:
# # CSV Loader

# loader = CSVLoader(file_path='cleaned_spotify_reviews.csv', source_column="review_text", encoding='MacRoman')
# docs = loader.load()

# # # Create a list of Document instances from the loaded documents
# # documents = [
# #     Document(
# #         page_content=doc['review_text'],  # Accessing the text content of the Document
# #         metadata={
# #             "review_rating": doc['review_rating'],
# #             "review_likes": doc['review_likes'],
# #             "author_app_version": doc['author_app_version']
# #         }
# #     )
# #     for doc in docs  # Iterate over the list of loaded Document objects
# ]

In [4]:
# from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# # Embeddings and Vector Database
# # instructor_embeddings = HuggingFaceInstructEmbeddings()
# # instructor_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# # vectordb = FAISS.from_documents(documents=docs, embedding=instructor_embeddings)

# instructor_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# # Initialize the text splitter with desired chunk size and overlap
# text_splitter = RecursiveCharacterTextSplitter(
#                     chunk_size=512, 
#                     chunk_overlap=24
# )

# df = pd.read_csv('cleaned_spotify_reviews.csv', encoding='MacRoman').dropna()

# # Initialize an empty list to hold documents
# all_documents = []

# # Use tqdm for progress bar
# for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
#     # Use the text splitter to split the review_text
#     chunks = text_splitter.split_text(row['review_text'])
    
#     # Create Document instances for each chunk
#     for chunk in chunks:
#         document = Document(
#             page_content=chunk,
#             metadata={
#                 "review_rating": row['review_rating'],
#                 "review_likes": row['review_likes'],
#                 "author_app_version": row['author_app_version']
#             }
#         )
#         all_documents.append(document)

# # Create FAISS index from all accumulated documents
# print("Creating FAISS index...")
# vectordb = FAISS.from_documents(documents=all_documents, embedding=instructor_embeddings)

**Creating the Vector Database from the CSV File**

In [4]:
# Setting up for the Faiss Vector Database
embeddings = "sentence-transformers/all-MiniLM-L6-v2"
file_path = "cleaned_spotify_reviews.csv"  
vectordb_file_path = "faiss_index"  # Save path for the FAISS vector database

In [22]:
# Function to ingest data into vector database
def ingest_to_vector_db(df, text_column, embedding_model):
    documents = [
        Document(
            page_content=row[text_column],
            metadata={
                "review_rating": float(row["review_rating"]),
                "review_likes": row["review_likes"],
                "author_app_version": row["author_app_version"]
            }
        )
        for _, row in df.iterrows()
    ]

    # Use the RecursiveCharacterTextSplitter to split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    document_chunks = text_splitter.split_documents(documents)

    # Create FAISS index from documents
    vectorstore_db = FAISS.from_documents(
        document_chunks,
        embedding_model,
        distance_strategy=DistanceStrategy.COSINE,
    )
    
    return vectorstore_db

In [23]:
# Function to load and process CSV
def load_and_process_csv(csv_path, text_column, embedding_model):
    df = pd.read_csv(csv_path, encoding='MacRoman')

    # Remove rows with NaN values in critical columns
    df = df[['review_text', 'review_rating', 'review_likes', 'author_app_version', 'review_timestamp']].dropna()

    # Ingest the cleaned data into the vector database
    vectordb = ingest_to_vector_db(df, text_column=text_column, embedding_model=embedding_model)
    
    # Save the vector database locally
    vectordb.save_local(vectordb_file_path)
    
    print(f"Vector database saved at '{vectordb_file_path}'")
    return vectordb

In [5]:
# Initialize the embeddings model
embedding_model = HuggingFaceEmbeddings(model_name=embeddings)

C:\Users\colaz\AppData\Local\Temp\ipykernel_9092\870974509.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=embeddings)
C:\Users\colaz\Projects\ML\LangChain\Mekari\Menv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\colaz\Projects\ML\LangChain\Mekari\Menv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This b

In [8]:
# Load and process CSV data
load_and_process_csv(file_path, text_column='review_text', embedding_model=embedding_model)

Error occurred: name 'load_and_process_csv' is not defined


In [6]:
# Creating the Retriever
vectordb = FAISS.load_local(vectordb_file_path, embedding_model, allow_dangerous_deserialization=True)
retriever = vectordb.as_retriever()

**Setting up the Chatbot**

In [7]:
# Initialize the memory for conversation
memory = ConversationBufferWindowMemory(k=5)  # Keep the last 5 interactions

C:\Users\colaz\AppData\Local\Temp\ipykernel_9092\1758680811.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=5)  # Keep the last 5 interactions


In [8]:
# Define the prompt template
prompt_template = """
You are an assistant designed to answer questions based on user reviews of a music streaming application.

**Layer 1: Contextual Understanding**
Please read the following user reviews carefully and provide precise answers to the questions based on the source document context (Spotify User Reviews Database) provided. Do not mention other companies.

**Layer 2: User Instruction**
Respond in a friendly and professional tone. If the answer involves comparisons to other music streaming platforms, provide specific examples from the source document context. 

**Layer 3: Response Guidelines**
If the answer is not explicitly found in the source document context, kindly state: "I'm sorry, I don't have that information." Please do not fabricate any answers or discuss unrelated topics.

CONTEXT: {context}

QUESTION: {question}
"""

# Create the PromptTemplate
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [9]:
# Setting up the Chain
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    output_key="result" 
)

In [10]:
# Function to analyze sentiment
def analyze_sentiment(review_text):
    analysis = TextBlob(review_text)
    sentiment_score = analysis.sentiment.polarity  # Returns a value between -1 (negative) and 1 (positive)

    # Create a more informative response based on the sentiment score
    if sentiment_score > 0.2:
        sentiment_description = "positive"
    elif sentiment_score < -0.2:
        sentiment_description = "negative"
    else:
        sentiment_description = "neutral"

    return f"Based on the review, the sentiment is {sentiment_description} with a score of {sentiment_score:.2f}."

# Function to summarize reviews
def summarize_reviews():
    summary_query = "Can you summarize the key points from the reviews?"
    summary_response = chain({"query": summary_query})
    return summary_response['result']

# Function to compare features
def compare_features(feature_1, feature_2):
    comparison_query = f"What do users say about {feature_1} compared to {feature_2}?"
    comparison_response = chain({"query": comparison_query})
    return comparison_response['result']

# Function to analyze trends
def analyze_trends():
    # Retrieve relevant documents from the vector database
    trends_query = "What trends are users discussing in their reviews?"
    # Process the query through the chain
    response = chain({"query": trends_query})

    # Save the context in memory after getting the response
    memory.save_context({"input": trends_query}, {"output": response['result']})  # Store the result using the output_key specified
    return response['result']

In [29]:
# Function to calculate cosine similarity score for quality scoring
def calculate_similarity_score(response_text, expected_text, embedding_model):
    response_embedding = embedding_model.embed_query(response_text)
    expected_embedding = embedding_model.embed_query(expected_text)
    similarity_score = cosine_similarity([response_embedding], [expected_embedding])[0][0]
    return similarity_score

# Function to score the response quality
def quality_scoring(response, expected_answer, embedding_model):
    relevance_score = calculate_similarity_score(response, expected_answer, embedding_model)
    accuracy_score = 1 if "factually correct" in response else 0.8
    clarity_score = 1 if len(response) > 50 else 0.7
    final_score = (0.5 * relevance_score) + (0.3 * accuracy_score) + (0.2 * clarity_score)
    return {
        "relevance_score": relevance_score,
        "accuracy_score": accuracy_score,
        "clarity_score": clarity_score,
        "final_score": final_score
    }

In [11]:
def handle_query(query):
    # Check if the query requests a summary, sentiment analysis, or feature comparison
    if "summarize" in query.lower():
        return summarize_reviews()
    elif "analyze sentiment for:" in query.lower():
        review_text = query.split("analyze sentiment for:")[-1].strip()
        return analyze_sentiment(review_text)
    elif "trends" in query.lower() or "patterns" in query.lower():
        return analyze_trends()
    else:
        # Process the query through the chain
        response = chain({"query": query})

        # Save the context in memory after getting the response
        memory.save_context({"input": query}, {"output": response['result']})  # Store the result using the output_key specified
        
        # If no result is found in the context, return a default message
        if "I'm sorry, I don't have that information." in response['result']:
            return "I'm sorry, I don't have information about that topic. Please ask about user reviews or features related to the topic."
        
        return response['result']

**Testing the Chatbot**

In [12]:
response = handle_query("What are the specific features or aspects that users appreciate the most in our application?")
print(response)

C:\Users\colaz\AppData\Local\Temp\ipykernel_9092\461798614.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({"query": query})



RESPONSE: Based on the user reviews, the personalized experience and the variety of features are the most appreciated aspects of our application. Users love the ability to customize their music experience and the wide range of features available to them. Some examples of these features include curated playlists, personalized recommendations, and the ability to explore new music.


In [24]:
response = handle_query("Can you summarize the reviews?")
print(response)


RESPONSE: From the reviews, it seems that users appreciate the ability to read specific parts of reviews and that all reviews are read. They also value the overall review system.


In [27]:
response = handle_query("In comparison to our application, which music streaming platform are users most likely to compare ours with?")
print(response)


RESPONSE: Based on user reviews, it seems that users are most likely to compare our application with other music streaming platforms such as Apple Music or Amazon Music. However, our user interface has been praised as the best among all music streaming apps.


In [14]:
response = handle_query("What are the primary reasons users express dissatisfaction with Spotify?")
print(response)


RESPONSE: Based on the user reviews, the primary reasons for dissatisfaction with Spotify seem to be the monetization of basic features and a lack of clear value in using the platform.


In [26]:
response = handle_query("Can you identify emerging trends or patterns in recent user reviews that may impact our product strategy?")
print(response)


The users are discussing changes and improvements being made to the reviews, as well as the frequency of advertisements on the platform.


**Checking for Hallucinations**

In [16]:
response = handle_query("What are the primary reasons users express dissatisfaction with Google?")
print(response)

I'm sorry, I don't have information about that topic. Please ask about user reviews or features related to the topic.


In [17]:
response = handle_query("Who won the world cup in 2034?")
print(response)

I'm sorry, I don't have information about that topic. Please ask about user reviews or features related to the topic.


In [22]:
response = handle_query("What color is your house?")
print(response)

I'm sorry, I don't have information about that topic. Please ask about user reviews or features related to the topic.


In [19]:
response = handle_query("What are the primary reasons users express dissatisfaction with Counter Strike?")
print(response)

I'm sorry, I don't have information about that topic. Please ask about user reviews or features related to the topic.


In [20]:
response = handle_query("What is the best features in Indofood?")
print(response)

I'm sorry, I don't have information about that topic. Please ask about user reviews or features related to the topic.


In [21]:
response = handle_query("What is Nvidia current stock price?")
print(response)

I'm sorry, I don't have information about that topic. Please ask about user reviews or features related to the topic.


In [281]:
chain("90890hubhjbhj")

{'query': '90890hubhjbhj',
 'result': "\nI'm sorry, I don't have that information. Could you please provide a question related to the user reviews for a music streaming application?",
 'source_documents': [Document(metadata={'review_rating': 5.0, 'review_likes': 0, 'author_app_version': '8.4.3.479'}, page_content='90200000'),
  Document(metadata={'review_rating': 5.0, 'review_likes': 0, 'author_app_version': '8.6.46.886'}, page_content='08022021'),
  Document(metadata={'review_rating': 1.0, 'review_likes': 0, 'author_app_version': '8.7.92.521'}, page_content='692023'),
  Document(metadata={'review_rating': 3.0, 'review_likes': 0, 'author_app_version': '8.6.44.851'}, page_content='118807990911_')]}